In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import expit

# 2D classification

In [ ]:
from project2_tools import get_2Ddata, to_onehot, from_onehot
from neuralnet import NeuralNet

In [ ]:
data_input, data_targets = get_2Ddata(t_crit = 2.3, onehot = True)

In [ ]:
from sklearn.model_selection import train_test_split
temp = train_test_split(data_input, data_targets, test_size = 0.33)
input_train, input_test, target_train, target_test = temp

## Setup network and run

In [ ]:
input_train.shape

In [ ]:
import pandas as pd

In [ ]:
# eta = 1e-2
n_batches = 10000

eta_values = 10**(np.arange(-4,0,dtype=np.float64))

# np.logspace(np.log10(5),2, 15, dtype=int)
n_hidden_values = [100]

In [ ]:
accuracies = []

In [ ]:
from IPython.display import clear_output
# n_hidden = 10

for eta in eta_values:
    for n_hidden in n_hidden_values:
        
        layer_sizes = [1600, n_hidden, 2]

        net = NeuralNet(layer_sizes, act_func = ['sigmoid', 'softmax'], net_type = 'classifier')
        min_epochs = 40
        max_epochs = 100

        tot = max_epochs * n_batches
        tol = 1e-10
        accuracy = []
        current = len(accuracy) + 1

        import time
        start = time.time()
        from project2_tools import batches

        inp =input_train
        tar = target_train

        for j in range(max_epochs):
            b = batches(input_train, target_train, n_batches = n_batches, zipped = False)

            for k, (x,y) in enumerate(b):
                if k%1000 == 0:
                    clear_output(wait = True)
                    print('n hidden nodes: ', n_hidden)
                    print('eta: ', eta)
                    print('batch size: ', x.shape[0])
                    print('batch, epoch, accuracy')
                    print('{:5}/{}  {:5}/{}  {:.2f}'.format(k, n_batches, j, max_epochs, accuracy[-1] if len(accuracy) else 0))#, 'o', markersize = 9)
                    now = time.time() 
                    print('Time estimate: {:.0f} seconds left'.format((now - start)/current * (tot-current)))
                net.update_batch_vectorized(x, y, eta)

                current += 1

            acc = net.accuracy(input_test, target_test)
            accuracy.append(acc)

            if len(accuracy) > min_epochs:
                cond1 = np.all(accuracy[-1] < np.array(accuracy[-5:-1]))  # decreasing accuracy
                cond2 = np.abs(accuracy[-1] - accuracy[-2])/accuracy[-1] < tol # stabilized accuracy
                if cond1 or cond2:
                    print('earlystop at epoch number {}'.format(len(accuracy)))
                    break

        accuracies.append({'eta':eta,'nhidden':layer_sizes[1], 'accuracy':accuracy, 
                           'epochs':len(accuracy),'n_batches':n_batches})

In [ ]:
df = pd.DataFrame(accuracies)
df['final_accuracy'] = df['accuracy'].str[-1]
df['max_accuracy'] = df['accuracy'].apply(np.max)
df

In [ ]:

layer_sizes = [input_train.shape[1], n_hidden,
                    target_train.shape[1]]
layer_sizes

def a(**kwargs):
    params = { 'n_batches':10000, 'min_epochs':40, 'max_epochs':100,
            'tol':1e-10, 'hidden_act_func':'sigmoid'}
    params.update(kwargs)
    tol = params.pop('tol')
    n_batches = params.pop('n_batches')
    min_epochs = params.pop('min_epochs')
    max_epochs = params.pop('max_epochs')
    hidden_act_func = params.pop('hidden_act_func')
        
    if params:
        raise TypeError("unrecognized kwarg(s): {}".format(
        ", ".join(params.keys())))



    pass
a(tol = 1)

In [ ]:
a = {'a':1}
b = {'b':2}

a.update(b)
a

In [ ]:
df.to_pickle

In [ ]:
df = pd.read_pickle('neural_net_run1.pickle')
df

In [ ]:
plt.figure(figsize = [16,9])
markers = ['--','-.','-',':']
colors = plt.cm.viridis(np.linspace(0,1,len(n_hidden_values)))


for (nhidden, item), c in zip(df.groupby('nhidden'),colors):
    
    print(nhidden)
    
    for (eta, eta_item),m in zip(item.groupby('eta'), markers):
        if True:#eta == 0.1:
            y = eta_item['accuracy'].values[0]
            plt.plot(y, label = 'eta = {:6.4f}, nhidden = {}'.format(eta, nhidden), c= c, linestyle = m)
        # print(y[-1])
    
plt.grid()
plt.legend()

In [ ]:
temp = df[['nhidden','eta','max_accuracy']]
temp = temp.set_index(['nhidden','eta'],drop=True).unstack()
temp.columns = temp.columns.droplevel(level=0)
temp

In [ ]:
ax = plt.gca()
m = ax.imshow(temp)

ax.set_yticks([0,1,2,3,4])
ax.set_yticklabels(temp.index)
ax.set_ylabel('Number of hidden nodes')

ax.set_xticks([0,1,2,3])
ax.set_xticklabels(temp.columns)
ax.set_xlabel('Eta')

plt.colorbar(m)

In [ ]:
ax = plt.gca()
temp2 = temp.iloc[:,2:]
m = ax.imshow(temp2)

ax.set_yticks([0,1,2,3,4])
ax.set_yticklabels(temp2.index)
ax.set_ylabel('Number of hidden nodes')

ax.set_xticks([0,1])
ax.set_xticklabels(temp2.columns)
ax.set_xlabel('Eta')

plt.colorbar(m)

In [ ]:
temp.plot.bar(rot = 0)

In [ ]:

speed = [0.1, 17.5, 40, 48, 52, 69, 88]
lifespan = [2, 8, 70, 1.5, 25, 12, 28]
index = ['snail', 'pig', 'elephant',
         'rabbit', 'giraffe', 'coyote', 'horse']
df = pd.DataFrame({'speed': speed,
                   'lifespan': lifespan}, index=index)
ax = df.plot.bar(rot=0)


In [ ]:
df

In [ ]:
plt.plot(accuracy)

plt.grid()

In [ ]:
print(np.abs(accuracy[-1] - accuracy[-2])/accuracy[-2])

In [ ]:
from IPython.display import clear_output
i = 0

eta = 1e-3
mse = []
n_epochs = 100
n_batches = 10000

tot = n_epochs * n_batches
current = 1

import time
start = time.time()

for j in range(n_epochs):
    b = batches(input_train, target_train, n_batches = n_batches)
    for k, batch in enumerate(b):
        clear_output(wait = True)
        print('batch, epoch, MSE')
        print('{:5}/{}  {:5}/{}  {:.2f}'.format(k, n_batches, j, n_epochs, mse[-1] if len(mse) else 0))#, 'o', markersize = 9)
        now = time.time() 
        print('Time estimate: {:.0f} seconds left'.format((now - start)/current * (tot-current)))
        net.update_batch(batch, eta)
    
        current += 1

    mse.append(np.average(test_net(input_test, target_test, net)))
    if np.isnan(mse[-1]) or np.any([np.any(np.isnan(w)) for w in net.weights]):
        print('NAN!!! Break! Abort mission!')
        break

In [ ]:
def f(x,y,z):
    return x+y+z

a = [1,2,3]

f(*a)

$\in$

In [ ]:
from project2_tools import softmax

%timeit net.feed_forward(input_train[0])


In [ ]:
net.backpropagate(input_train[0], target_train[0])

In [ ]:
plt.plot(mse)

In [ ]:
from nielsen_network import Network as nNetwork

net2 = nNetwork(layer_sizes)
net2.backprop(np.atleast_2d(x),np.atleast_2d(y))
# x,y = batch[0]

In [ ]:
np.atleast_2d(x).shape

In [ ]:
y

find last delta:
$$ \delta_j^L = f'(z_j^L) \frac{\partial \mathcal{C}}{\partial a_j^L} $$

propagate deltas for each layer
$$ \delta_j^l = \sum_k \delta_k^{l+1} w_{kj}^{l+1} f'(z_j^l)$$

update weights and bias
$$w_{jk}^l \leftarrow w_{jk}^l - \eta \delta_j^l a_k^{l-1} $$
$$b_{j}^l \leftarrow b_{j}^l - \eta \delta_j^l  $$



we have

$$ \frac{\partial \mathcal{C}}{\partial a_j^L} = a_j^L - t_j, $$
and
$$ f'(z_j^l) = f(z_j^l)  (1 - f(z_j^l) $$
for sigmoid

In [ ]:
for i, batch in enumerate(batches(input_train, target_train, n_batches = 100)):
    break
    

In [ ]:
x,y = batch[0]
x,y

In [ ]:
net = NeuralNet(layer_sizes, act_func = ['sigmoid','identity'])

for k in range(20):
    for i, batch in enumerate(batches(input_train, target_train, n_batches = 100)):
        net.update_batch(batch, eta = 1e-2)

    plt.plot(k, np.average(test_net(input_test, target_test, net)), 'o', markersize = 10)

In [ ]:
fig,[ax1,ax2] = plt.subplots(1,2, figsize = [8,4])
i = 43
test_out = np.array([net.feed_forward(inp)[1][-1] for inp in input_test]).squeeze()
ax1.scatter(test_out, target_test)
ax1.set_xlabel('predicted')
ax1.set_ylabel('target')

ax1.plot([-12,12],[-12,12])
ax1.axis('equal')


#ax2.hist(test_out,  alpha = 0.5)
#ax2.hist(target_test,  alpha = 0.5);